# Navigation Pixels

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.nn import Conv2d, MaxPool2d, Conv3d

class DuelingQNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, m=4):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(DuelingQNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        
        in_channels = state_size[0]
        self.conv1 = Conv2d(in_channels, 1, kernel_size=(1,1), stride=(1,1), padding=(0,0))
        self.conv2 = Conv2d(1, 8, kernel_size=(8,8), stride=(4,4), padding=(0,0))
        self.conv3 = Conv2d(8, 16, kernel_size=(4,4), stride=(2,2), padding=0)
        self.conv4 = Conv2d(16, 32, kernel_size=(4,4), stride=(2,2), padding=0)
        self.fc5 = nn.Linear(3*3*32, 16)
        self.fc_v = nn.Linear(16, 1)
        self.fc_a = nn.Linear(16, action_size)    

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.conv1(state))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 3*3*32)
        x = F.relu(self.fc5(x))
        V = self.fc_v(x)
        A = self.fc_a(x)
        A = A - A.mean(1).unsqueeze(1)
        Q = V + A
        return Q

In [3]:
import sys
sys.path.append('..')
from baselines.common.segment_tree import *
from baselines.deepq.replay_buffer import ReplayBuffer, PrioritizedReplayBuffer

import numpy as np
import random
from collections import namedtuple, deque

BUFFER_SIZE = int(3e4)         # replay buffer size
BATCH_SIZE = 64                # minibatch size
GAMMA = 0.99                   # discount factor
TAU = 1e-3                     # for soft update of target parameters
LR = 5e-3                      # learning rate 
UPDATE_EVERY = 4               # how often to update the network
PRIORITIZED_REPLAY_EPS = 1e-7  # prioritized replay eps

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class LinearSchedule():
    
    def __init__(self, vals, steps):
        assert len(steps) == len(vals)
        for step in steps:
            assert isinstance(step, int)
            assert step >= 0
        assert steps[0] == 0
        for i in range(1,len(steps)):
            assert steps[i] > steps[i-1]
            
        self.vals = vals
        self.steps = steps
        self.eps = 0
        self.findex = 0
        self.f = []
        for i in range(1,len(self.steps)):
            s1, s0 = self.steps[i], self.steps[i-1]
            v1, v0 = self.vals[i], self.vals[i-1]
            ramp = (v1-v0)/(s1-s0)
            self.f.append(lambda e, ramp=ramp, s0=s0, v0=v0: v0+(e-s0)*ramp)            
        self.f.append(lambda e: self.vals[-1])
        
    def __call__(self, done=False):
        val = self.f[self.findex](self.eps)
        if done:
            self.eps += 1
            if self.findex < len(self.f)-1:
                if self.eps >= self.steps[self.findex+1]:
                    self.findex += 1
        return val


class DoubleDuelingQAgent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = DuelingQNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = DuelingQNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = PrioritizedReplayBuffer(BUFFER_SIZE, 1e-5)
        self.memory._max_priority = 1e-5
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
        # Schedules for alpha and beta parameters
        self.alpha = LinearSchedule([0.00], [0])        
        self.beta = LinearSchedule([0.001], [0])        
    
    def step(self, state, action, reward, next_state, done):

        idx = self.memory._next_idx
        
        # Add sample buffer to replay memory
        self.memory.add(state, action, reward, next_state, done)
        states = np.expand_dims(state, axis=0)
        actions = np.expand_dims(np.expand_dims(action, axis=0), axis=-1)
        rewards = np.expand_dims(reward, axis=0)        
        next_states = np.expand_dims(next_state, axis=0)        
        dones = np.expand_dims(done, axis=0)
        
        delta = self.compute_delta(states, actions, rewards, next_states, dones, GAMMA)
        priorities = np.abs(np.squeeze(delta.cpu().detach().numpy(), axis=0)) + PRIORITIZED_REPLAY_EPS
        self.memory.update_priorities([idx], priorities)        
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
          
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample(BATCH_SIZE, self.beta(done))
                self.memory.alpha = self.alpha(done)
                self.learn(experiences, GAMMA)
                
    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            actions_local = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(actions_local.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
    
    def compute_delta(self, states, actions, rewards, next_states, dones, gamma):
        states = torch.from_numpy(states).float().to(device)
        actions = torch.from_numpy(actions).long().to(device)
        rewards = torch.from_numpy(rewards).float().to(device)
        next_states = torch.from_numpy(next_states).float().to(device)
        dones = torch.from_numpy(dones.astype(np.uint8)).float().to(device)
        
        # Get max predicted Q values (for next states) from target model
        # Double DQN
        actions_next_state = self.qnetwork_local(states).detach().argmax(1).unsqueeze(1)
        Q_targets_next = self.qnetwork_target(next_states).detach().gather(1, actions_next_state)
        # DQN
        # Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss
        delta = Q_targets - Q_expected
        
        return delta

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """        
        states, actions, rewards, next_states, dones, weights, indices = experiences
        
        actions = np.expand_dims(actions, axis=-1)
        rewards = np.expand_dims(rewards, axis=-1)
        dones = np.expand_dims(dones, axis=-1)
        weights = np.expand_dims(weights, axis=-1)
        states = torch.from_numpy(states).float().to(device)
        actions = torch.from_numpy(actions).long().to(device)
        rewards = torch.from_numpy(rewards).float().to(device)
        next_states = torch.from_numpy(next_states).float().to(device)
        dones = torch.from_numpy(dones.astype(np.uint8)).float().to(device)
        weights = torch.from_numpy(weights).float().to(device)

        # Get max predicted Q values (for next states) from target model
        # Double DQN
        actions_next_state = self.qnetwork_local(states).detach().argmax(1).unsqueeze(1)
        Q_targets_next = self.qnetwork_target(next_states).detach().gather(1, actions_next_state)
        # DQN
        # Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss
        delta = Q_targets - Q_expected
        squared_difference = delta ** 2
        weighted_squared_difference = weights * squared_difference
        loss = weighted_squared_difference.mean()
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU) 

        # ------------------- update priorities ------------------- #  
        new_priorities = np.abs(np.squeeze(delta.cpu().detach().numpy())) + PRIORITIZED_REPLAY_EPS
        self.memory.update_priorities(indices, new_priorities)

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [4]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="VisualBanana_Linux/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 1
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 0
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [5]:
def frame2state(frame):
    N, H, W, C = frame.shape
    state = np.squeeze(frame)
    state = np.rollaxis(state, 2, 0)
    return state

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of actions
action_size = brain.vector_action_space_size

# examine the state space 
state = env_info.visual_observations[0]
state_size = (state.shape[3], state.shape[1], state.shape[2])
frame_size = (state.shape[3]+2, 1, state.shape[1], state.shape[2])

agent = DoubleDuelingQAgent(state_size=state_size, action_size=action_size, seed=0)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import time

def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        frame = env_info.visual_observations[0]            # get the current state
        state = frame2state(frame)
        
        score = 0
        for t in range(max_t):
            
            action = agent.act(state, eps) # select an action
            env_info = env.step(action)[brain_name]            # send the action to the environment
            next_frame = env_info.visual_observations[0]       # get the next state
            next_state = frame2state(next_frame)
            reward = env_info.rewards[0]                       # get the reward
            done = env_info.local_done[0]                      # see if episode has finished
            
            start_time = time.time()
            agent.step(state, action, reward, next_state, done)  
            #print('step', time.time()-start_time)  
            
            state = next_state

            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        if np.mean(scores_window)>=1.0:
            eps = max(eps_end, eps_decay*eps) # decrease epsilon
        else:
            eps = max(0.65, eps_decay*eps)
        print('\rEpisode {}\tAverage Score: {:.2f} '.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

import time
start = time.time()
scores = dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995)
end = time.time()
print('time = {}'.format(end-start))

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')

# Close the environment
env.close()

Episode 100	Average Score: -0.08 
Episode 200	Average Score: -0.08 
Episode 300	Average Score: -0.14 
Episode 400	Average Score: 0.03  
Episode 500	Average Score: 0.12  
Episode 566	Average Score: -0.08 